# Conceptos de ETL y ELT

## Objetivos de Aprendizaje
- Entender qué es un pipeline de datos
- Diferenciar entre ETL y ELT
- Conocer los componentes de un pipeline
- Identificar cuándo usar cada enfoque

## Prerequisitos
- `00_setup/02_spark_basics.ipynb`

## Tiempo Estimado
⏱️ 30 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 6: Data Processing and Analysis - ETL concepts

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime

spark = SparkSession.builder.appName("ETL_Concepts").getOrCreate()
spark.sparkContext.setLogLevel("WARN")
print("Spark listo para ETL")

---
# === SECCIÓN 1 ===
## 1. ¿Qué es un Pipeline de Datos?

### Explicación Conceptual

Un **pipeline de datos** es un conjunto de procesos que mueven datos desde una o más fuentes hasta uno o más destinos, aplicando transformaciones en el camino.

**Analogía del mundo real:** Imagina una fábrica de jugo de naranja:
- **Extract**: Las naranjas llegan de diferentes granjas (fuentes)
- **Transform**: Se lavan, pelan, exprimen y filtran (procesamiento)
- **Load**: El jugo se embotella y distribuye (destino)

En datos:
- **Extract**: Leer datos de bases de datos, APIs, archivos
- **Transform**: Limpiar, normalizar, enriquecer, agregar
- **Load**: Guardar en data warehouse, data lake, base de datos

In [ ]:
# Ejemplo simple de pipeline

# 1. EXTRACT - Simular datos de fuente
datos_crudos = [
    ("001", "  Juan Pérez  ", "CDMX", "2024-01-15", 1500.50),
    ("002", "maria garcia", "gdl", "2024/01/16", 2300.00),
    ("003", "CARLOS LOPEZ", "MTY", "15-01-2024", None),  # Valor nulo
    ("002", "maria garcia", "gdl", "2024/01/16", 2300.00),  # Duplicado
]

df_raw = spark.createDataFrame(
    datos_crudos, 
    ["id", "nombre", "ciudad", "fecha", "monto"]
)

print("DATOS CRUDOS (Extract):")
df_raw.show(truncate=False)

In [ ]:
# 2. TRANSFORM - Limpiar y normalizar

df_transformed = df_raw \
    .dropDuplicates(["id"]) \
    .withColumn("nombre", F.initcap(F.trim(F.col("nombre")))) \
    .withColumn("ciudad", F.upper(F.col("ciudad"))) \
    .withColumn("monto", F.coalesce(F.col("monto"), F.lit(0.0))) \
    .withColumn("fecha_proceso", F.current_timestamp())

print("DATOS TRANSFORMADOS (Transform):")
df_transformed.show(truncate=False)

In [ ]:
# 3. LOAD - Guardar (simulado)

# En un pipeline real, aqui guardariamos a Parquet, base de datos, etc.
# df_transformed.write.mode("overwrite").parquet("/data/processed/clientes")

print("DATOS LISTOS PARA CARGA (Load):")
print(f"  Registros: {df_transformed.count()}")
print(f"  Columnas: {df_transformed.columns}")
df_transformed.printSchema()

---
# === SECCIÓN 2 ===
## 2. ETL vs ELT

### Explicación Conceptual

**ETL (Extract-Transform-Load)**
- Transforma los datos ANTES de cargarlos al destino
- El procesamiento ocurre en un servidor ETL dedicado
- Tradicional, usado con data warehouses on-premise

**ELT (Extract-Load-Transform)**
- Carga los datos crudos primero, transforma después
- El procesamiento ocurre en el destino (data lake/warehouse)
- Moderno, aprovecha poder de cómputo en la nube

**¿Cuándo usar cada uno?**

| Criterio | ETL | ELT |
|----------|-----|-----|
| Volumen de datos | Pequeño-Mediano | Grande |
| Destino | Data Warehouse tradicional | Data Lake / Cloud DW |
| Cómputo | Limitado en destino | Escalable en destino |
| Latencia | Mayor | Menor (carga rápida) |
| Flexibilidad | Menos (esquema fijo) | Más (esquema on-read) |

In [ ]:
# Ejemplo conceptual: ETL tradicional
print("=== ENFOQUE ETL ===")
print()
print("1. Extract: Leer ventas del día")
print("2. Transform EN SPARK:")
print("   - Limpiar datos")
print("   - Calcular totales")
print("   - Agregar por región")
print("3. Load: Guardar resumen en Data Warehouse")
print()
print("Ventaja: Solo datos limpios llegan al DW")
print("Desventaja: Si necesitas los datos crudos, no los tienes")

In [ ]:
# Ejemplo conceptual: ELT moderno
print("=== ENFOQUE ELT ===")
print()
print("1. Extract: Leer ventas del día")
print("2. Load: Guardar datos CRUDOS en Data Lake (S3)")
print("3. Transform EN EL DATA LAKE:")
print("   - Crear vista limpia")
print("   - Crear vista agregada")
print("   - Mantener datos crudos disponibles")
print()
print("Ventaja: Flexibilidad, datos crudos siempre disponibles")
print("Desventaja: Requiere más almacenamiento")

---
# === SECCIÓN 3 ===
## 3. Componentes de un Pipeline ETL

### Explicación Conceptual

Un pipeline robusto incluye:
1. **Fuentes de datos**: Bases de datos, archivos, APIs
2. **Validación de entrada**: Verificar que los datos llegaron
3. **Transformaciones**: Limpieza, enriquecimiento
4. **Validación de salida**: Verificar calidad de datos
5. **Carga**: Escribir al destino
6. **Logging**: Registrar qué pasó
7. **Manejo de errores**: Qué hacer si algo falla

In [ ]:
# Ejemplo de pipeline estructurado

class ETLPipeline:
    """Pipeline ETL simple para demostración"""
    
    def __init__(self, spark, nombre):
        self.spark = spark
        self.nombre = nombre
        self.logs = []
    
    def log(self, mensaje):
        """Registrar evento en el pipeline"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        log_entry = f"[{timestamp}] {self.nombre}: {mensaje}"
        self.logs.append(log_entry)
        print(log_entry)
    
    def extract(self, datos, schema):
        """Fase de extracción"""
        self.log("Iniciando extracción...")
        df = self.spark.createDataFrame(datos, schema)
        self.log(f"Extraídos {df.count()} registros")
        return df
    
    def validate(self, df, reglas):
        """Validar datos según reglas"""
        self.log("Validando datos...")
        errores = []
        
        for columna, regla in reglas.items():
            if regla == "not_null":
                nulos = df.filter(F.col(columna).isNull()).count()
                if nulos > 0:
                    errores.append(f"{columna}: {nulos} nulos")
        
        if errores:
            self.log(f"Advertencias: {errores}")
        else:
            self.log("Validación exitosa")
        
        return df
    
    def transform(self, df, transformaciones):
        """Aplicar transformaciones"""
        self.log("Aplicando transformaciones...")
        
        for nombre_t, func in transformaciones.items():
            df = func(df)
            self.log(f"  - Aplicada: {nombre_t}")
        
        return df
    
    def load(self, df, destino):
        """Cargar datos (simulado)"""
        self.log(f"Cargando a {destino}...")
        # En producción: df.write.mode("overwrite").parquet(destino)
        self.log(f"Cargados {df.count()} registros")
        return df

print("Clase ETLPipeline definida")

In [ ]:
# Usar el pipeline

# Datos de ejemplo
datos_ventas = [
    ("V001", "P001", 100.0, "2024-01-15"),
    ("V002", "P002", None, "2024-01-15"),
    ("V003", "P001", 150.0, "2024-01-16"),
]

# Transformaciones a aplicar
transformaciones = {
    "rellenar_nulos": lambda df: df.fillna({"monto": 0.0}),
    "agregar_timestamp": lambda df: df.withColumn("procesado", F.current_timestamp())
}

# Ejecutar pipeline
pipeline = ETLPipeline(spark, "VentasPipeline")

df = pipeline.extract(datos_ventas, ["venta_id", "producto_id", "monto", "fecha"])
df = pipeline.validate(df, {"monto": "not_null"})
df = pipeline.transform(df, transformaciones)
df = pipeline.load(df, "/data/processed/ventas")

print("\nResultado final:")
df.show()

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio ETL.1: Identificar Tipo de Pipeline

Para cada escenario, indica si usarías ETL o ELT y por qué:

1. Migrar datos de Oracle a Redshift (cloud)
2. Cargar logs de servidores a HDFS para análisis ad-hoc
3. Alimentar un dashboard de ventas diarias
4. Construir un data lake para machine learning

In [ ]:
# TODO: Escribe tus respuestas

respuestas = {
    "escenario_1": "___",  # ETL o ELT?
    "escenario_2": "___",
    "escenario_3": "___",
    "escenario_4": "___"
}

### ✅ Solución Ejercicio ETL.1

In [ ]:
respuestas = {
    "escenario_1": "ELT - Redshift es un cloud DW con poder de cómputo",
    "escenario_2": "ELT - Data lake, queremos datos crudos para análisis flexible",
    "escenario_3": "ETL - Dashboard necesita datos ya procesados y resumidos",
    "escenario_4": "ELT - ML necesita acceso a datos crudos para feature engineering"
}

for escenario, respuesta in respuestas.items():
    print(f"{escenario}: {respuesta}")

### 🎯 Ejercicio ETL.2: Mini Pipeline

Crea un pipeline que:
1. Extraiga datos de clientes con campos: id, email, pais
2. Transforme: emails a minúsculas, países a mayúsculas
3. Valide: que no haya emails nulos
4. Agregue timestamp de proceso

In [ ]:
# TODO: Crea tu pipeline

datos_clientes = [
    (1, "ANA@Email.COM", "mexico"),
    (2, "carlos@TEST.com", "colombia"),
    (3, None, "argentina"),
]

# Tu código aquí


### ✅ Solución Ejercicio ETL.2

In [ ]:
datos_clientes = [
    (1, "ANA@Email.COM", "mexico"),
    (2, "carlos@TEST.com", "colombia"),
    (3, None, "argentina"),
]

# Extract
df_clientes = spark.createDataFrame(datos_clientes, ["id", "email", "pais"])
print("1. EXTRACT:")
df_clientes.show()

# Validate
nulos = df_clientes.filter(F.col("email").isNull()).count()
print(f"2. VALIDATE: {nulos} emails nulos encontrados")

# Transform
df_transformado = df_clientes \
    .filter(F.col("email").isNotNull()) \
    .withColumn("email", F.lower(F.col("email"))) \
    .withColumn("pais", F.upper(F.col("pais"))) \
    .withColumn("fecha_proceso", F.current_timestamp())

print("3. TRANSFORM + 4. LOAD (listo):")
df_transformado.show(truncate=False)

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **Pipeline de datos**: Flujo de Extract → Transform → Load
- **ETL**: Transforma antes de cargar, tradicional
- **ELT**: Carga primero, transforma después, moderno/cloud
- **Componentes**: Fuentes, validación, transformación, carga, logging, errores

### Conexión con AWS
- **AWS Glue**: Servicio ETL serverless
- **Glue Studio**: ETL visual (drag & drop)
- **Glue Crawlers**: Descubren esquemas automáticamente
- **Step Functions**: Orquestar pipelines complejos

### Siguiente Paso
Continúa con: `02_data_extraction.ipynb` para aprender extracción de datos